In [1]:
from xml.dom import minidom
from unidecode import unidecode
import sys
import pandas as pd
sys.path.insert(1, '../utils')
from read_config import read_config_file

In [2]:
inputs_path = '../inputs/'
results_path = '../results/'

## Read TXT config file

In [3]:
config_dict = {}
read_config_file(inputs_path + 'PC.CFG', config_dict)
config_dict

{'LEIA': ['cfquery.xml'],
 'CONSULTAS': ['consultas.csv'],
 'ESPERADOS': ['esperados.csv']}

## Read XML

- parse xml

In [11]:
xml_file = inputs_path + config_dict['LEIA'][0]
parsed_xml = minidom.parse(xml_file)
query_elements = parsed_xml.getElementsByTagName('QUERY')

- Generate Queries CSV

In [15]:
xml_data_queries = []
for query in query_elements:
    query_number = query.getElementsByTagName('QueryNumber')[0].firstChild.nodeValue
    query_text = query.getElementsByTagName('QueryText')[0].firstChild.nodeValue.strip()
    query_text = unidecode(query_text.replace(';', ' ')).lower()
    xml_data_queries.append([query_number, query_text])

df_queries = pd.DataFrame(xml_data_queries, columns=['QueryNumber', 'QueryText'])

df_queries.to_csv(results_path + config_dict['CONSULTAS'][0], index=False, sep=';')

In [16]:
df_queries.head()

,QueryNumber,QueryText
0,00001,what are the effects of calcium on the physica...
1,00002,can one distinguish between the effects of muc...
2,00003,how are salivary glycoproteins from cf patient...
3,00004,what is the lipid composition of cf respirator...
4,00005,is cf mucus abnormal?


- Generate Expected Csv

In [21]:
# Extrai as informações do XML
xml_data_docs = []
for query in parsed_xml.getElementsByTagName('QUERY'):
    query_number = query.getElementsByTagName('QueryNumber')[0].firstChild.nodeValue
    for record in query.getElementsByTagName('Records')[0].getElementsByTagName('Item'):
        doc_number = record.firstChild.nodeValue
        doc_votes = record.getAttribute('score')
        xml_data_docs.append((query_number, doc_number, doc_votes))

# Cria um DataFrame com as informações extraídas do XML
df_docs = pd.DataFrame(xml_data_docs, columns=['QueryNumber', 'DocNumber', 'DocVotes'])
df_docs.to_csv(results_path + config_dict['ESPERADOS'][0], index=False, sep=';')

In [22]:
df_docs.head()

,QueryNumber,DocNumber,DocVotes
0,00001,139,1222
1,00001,151,2211
2,00001,166,0001
3,00001,311,0001
4,00001,370,1010
